In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
import seaborn as sns

#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('universal_tagset')
#nltk.download('vader_lexicon')

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
tokenizer.model_max_length = 512

skyscanner = pd.read_csv("skyskanner.csv")

skyscanner = skyscanner.dropna(subset=['review'])
shortStringCondition = skyscanner['review'].str.len() > 50 #Keeping only the 50 char + strings
skyscanner_long = skyscanner[shortStringCondition]
# Create empty columns to store the outputs
skyscanner_long['roberta_neg'] = ""
skyscanner_long['roberta_neu'] = ""
skyscanner_long['roberta_pos'] = ""



/var/folders/wf/mk3xyd5533b95_xs35xywm480000gn/T/ipykernel_33050/2636168903.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skyscanner_long['roberta_neg'] = ""
/var/folders/wf/mk3xyd5533b95_xs35xywm480000gn/T/ipykernel_33050/2636168903.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skyscanner_long['roberta_neu'] = ""
/var/folders/wf/mk3xyd5533b95_xs35xywm480000gn/T/ipykernel_33050/2636168903.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

FileNotFoundError: [Errno 2] No such file or directory: 'your_file.csv'

In [10]:
df = pd.read_csv("skyskanner_roberted.csv")

# Filter the DataFrame to keep only rows where roberta_neg, roberta_pos, and roberta_neu are null
df_filtered = df[df['roberta_neg'].isnull() & df['roberta_pos'].isnull() & df['roberta_neu'].isnull()]
skyscanner_long = df_filtered

skyscanner_long.to_csv("skyskanner_not_roberted")

In [12]:
skyscanner_long = pd.read_csv("skyskanner_not_roberted.csv")

In [13]:
length = len(skyscanner_long)
chunk_size = 10
onepercent = (length/100).__floor__()
print(length)
print(onepercent)
currpercent = 1
for i in range(0,length , chunk_size):
    chunk_tokenized = tokenizer(skyscanner_long["review"].iloc[i:i+chunk_size].tolist(), return_tensors='pt',padding='max_length', truncation=True, max_length=514)
    chunk_modeled = model(**chunk_tokenized)
    skyscanner_long.iloc[i:i+chunk_size, skyscanner_long.columns.get_loc('roberta_neg')] = chunk_modeled.logits[:, 0].detach().numpy()
    skyscanner_long.iloc[i:i+chunk_size, skyscanner_long.columns.get_loc('roberta_neu')] = chunk_modeled.logits[:, 1].detach().numpy()
    skyscanner_long.iloc[i:i+chunk_size, skyscanner_long.columns.get_loc('roberta_pos')] = chunk_modeled.logits[:, 2].detach().numpy()
    if (i / onepercent) > currpercent:
        print(i/onepercent)
        currpercent+=1

8965
89
1.0112359550561798
2.0224719101123596
3.033707865168539
4.044943820224719
5.056179775280899
6.067415730337078
7.078651685393258
8.089887640449438
9.101123595505618
10.112359550561798
11.01123595505618
12.02247191011236
13.03370786516854
14.044943820224718
15.0561797752809
16.06741573033708
17.078651685393258
18.089887640449437
19.10112359550562
20.1123595505618
21.01123595505618
22.02247191011236
23.03370786516854
24.04494382022472
25.0561797752809
26.06741573033708
27.078651685393258
28.089887640449437
29.10112359550562
30.1123595505618
31.01123595505618
32.02247191011236
33.03370786516854
34.04494382022472
35.056179775280896
36.06741573033708
37.07865168539326
38.08988764044944
39.10112359550562
40.1123595505618
41.01123595505618
42.02247191011236
43.03370786516854
44.04494382022472
45.056179775280896
46.06741573033708
47.07865168539326
48.08988764044944
49.10112359550562
50.1123595505618
51.01123595505618
52.02247191011236
53.03370786516854
54.04494382022472
55.0561797752808

In [23]:
skyscanner_long.to_csv('skyskanner_roberted2.csv', index=False)

In [28]:
sky1 = pd.read_csv("skyskanner_roberted.csv")
sky2 = pd.read_csv("skyskanner_roberted2.csv")
sky2 = sky2.drop(columns=['todel'])

sky1 = sky1.dropna(subset=['roberta_neg', 'roberta_pos', 'roberta_neu'])

In [29]:
combined = pd.concat([sky1, sky2], ignore_index=True)
combined

,year,rating,review,roberta_neg,roberta_neu,roberta_pos,Unnamed: 0
0,2023,5,This app is great! But I have a suggestion. It...,-2.366167,0.041758,2.773424,NaN
1,2021,3,"Upon updating the app, it appears that a lot o...",1.427508,0.138611,-1.749099,NaN
2,2020,1,The latest version as of 25DEC2020 is absolute...,2.564242,-0.412785,-2.259642,NaN
3,2020,1,The New View is Terrible! Primarily for esthet...,0.817050,0.057151,-0.882952,NaN
4,2021,1,"Years ago, I found great flights on this app. ...",-0.748656,-0.263892,1.354573,NaN
...,...,...,...,...,...,...,...
17690,2018,5,"My vacation always starts with skyscanner, esp...",-2.223770,1.319033,1.075548,17692.0
17691,2016,4,Only just opened Skyscanner and I think it's j...,-2.540681,0.442104,2.425169,17693.0
17692,2017,5,Already booked two of my cheapest flights from...,-3.408723,1.071380,2.481618,17694.0
17693,2017,5,This app help me a lot to find tickets for air...,-3.069469,0.547620,2.813903,17695.0


In [ ]:
combined = combined.drop(columns=['Unnamed: 0'])


In [ ]:
combined.to_csv("skyskanner_final.csv")
